# Imports

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import folium
from shapely.ops import unary_union
import matplotlib.pyplot as plt
from shapely import wkt

# Constants

In [2]:
CRS = 4326
CRS_METERS = 32616
COLORS = {
    'green': '#83bca9',
    'darkgreen': '#3e5641'
}

## Load in USA Parks data

In [3]:
usa_parks = gpd.read_file("../data/USA_Parks/v10/park_dtl.gdb/")
usa_parks = usa_parks.to_crs(epsg=CRS) # convert to const CRS value (32616)

## Load US State Boundaries

In [4]:
us_states = pd.read_csv("../data/United_States_Boundary_Files.csv")
us_states['geometry'] = us_states['the_geom'].apply(wkt.loads) # create geometry column with safety
us_states_gdf = gpd.GeoDataFrame(us_states, crs = CRS) # convert to GeoPd DF with const CRS value (4326)

In [32]:
# us_states_gdf.loc[us_states_gdf['STUSPS'] == 'TN'].to_file('../data/tn_state_border.geojson', driver='GeoJSON')

### Find parks only in or bordering TN

In [5]:
tn_parks = gpd.sjoin(usa_parks, # sjoin to find the parks that 'intersect' or are in and on the border of TN
                     us_states_gdf.loc[us_states_gdf['STUSPS'] == 'TN'],
                     predicate = 'intersects')

tn_parks['name'] = tn_parks['NAME_left'] # rename name column
tn_parks = tn_parks[['name', 'FEATTYPE', 'SQMI', 'geometry']] # keep only neccessary columns

In [6]:
# tn_parks.to_file('../data/tn_parks.geojson', driver='GeoJSON')

In [7]:
# tn_parks.shape[0]

#### 722 State and National Parks in (or bordering) Tennessee.

In [8]:
# tn_parks['FEATTYPE'].unique()

#### Types of Parks (5): Local park, State park or forest, County park, Regional park, National park or forest

In [9]:
# tn_parks.groupby('FEATTYPE')['name'].count()

In [10]:
# tn_parks.groupby('FEATTYPE')['SQMI'].median()

### Median Square Mile by Type
#### County park                 0.055
#### Local park                  0.030
#### National park or forest    36.180
#### Regional park               6.500
#### State park or forest        1.710

### Number of Parks by type
#### County park                 18
#### Local park                 571
#### National park or forest     28
#### Regional park               32
#### State park or forest        73

# Create a map of the TN parks

In [11]:
# tn_map = folium.Map(tiles = "Cartodb Positron",
#                      location = (36, -86.1),
#                      zoom_start = 7,
#                      prefer_canvas=True)

In [12]:
# add feature group
# tn_layer = folium.FeatureGroup(name = 'Tennessee Parks', show = True)

# for i, park in tn_parks.iterrows():    # iterate throug rows of TN parks GeoDF
#     textbox = folium.Popup(
#             park['name']
#     )
    
#     folium.GeoJson(
#             park['geometry'],
#             popup = textbox,
#             style_function=lambda x: {"fillColor": c['green'],
#                                                                  'color':COLORS['darkgreen'],
#                                                                 'weight': 0.5}).add_to(tn_layer)

# tn_layer.add_to(tn_map)
# folium.LayerControl().add_to(tn_map)

# Load in Places Data

In [13]:
places_df = pd.read_csv('../data/PLACES__Local_Data_for_Better_Health__Place_Data_2023_release_20240504.csv')
# data_dict = pd.read_csv('../data/PLACES_and_500_Cities__Data_Dictionary_20240504.csv')

#### Create places geopandas DF 

In [14]:
places_df['geometry'] = places_df['Geolocation'].apply(wkt.loads) # create geometry column with safety
places_gdf = gpd.GeoDataFrame(places_df, crs = CRS) # convert to GeoPd DF with const CRS value (4326)

### Find Places data only in TN and filter out unneccessary columns

In [15]:
tn_places = places_gdf.loc[places_gdf['StateAbbr'] == 'TN'][['Year',
                                                             'Category', 'Measure',
                                                             'Data_Value', 'TotalPopulation',
                                                            'geometry', 'LocationID', 'MeasureId']]

In [16]:
# tn_places.to_file('../data/tn_health_points.geojson', driver='GeoJSON')

In [17]:
# tn_places['Year'].agg(['min', 'max'])

#### years 2020, 2021

In [18]:
# tn_places.shape

#### 31,746 entries

In [19]:
# tn_places['LocationID'].nunique()

#### 429 different locations

####  Find the min distance between each Health data point and the nearest park. Save as CSV

In [20]:
# reproject to get distances in meters
# tn_health_loc = tn_places[['LocationID', 'geometry']].to_crs(epsg=CRS_METERS) # create smaller Df to iterate through
# tn_parks_loc = tn_parks[['name', 'geometry']].to_crs(epsg=CRS_METERS) # create smaller Df to iterate through

In [21]:
# min_dist = [] # empty list to hold the min distances to create a new column out of
# for i, r_health in tn_health_loc.iterrows(): # iterate through each row of the TN health data points
#     distances = [] # create empty list to hold the distnace from the 
#                    # health point and each state park boundary
#     for idx, r_park in tn_parks_loc.iterrows(): # iterate though each row of the TN parks data
#         distances.append(r_health['geometry'].distance(r_park['geometry'])) # add the distance from 
#                                                                             # the health point top
#                                                                             # each park to the list
#     min_dist.append(np.min(distances)) # find the distance from the health point to the nearest park

# tn_health_loc['distance'] = min_dist # create new column in the TN health points DF
# tn_health_loc.to_csv('../data/TN_health_data_with_distances_to_parks.csv') # save as csv for future recals

# Load in TN Helath Distances CSV

In [22]:
tn_health_distances = pd.read_csv('../data/TN_health_data_with_distances_to_parks.csv')

In [23]:
tn_places = tn_places.to_crs(epsg=CRS_METERS)
tn_places_distances = tn_places.merge(right = tn_health_distances[['LocationID', 'distance']], on='LocationID', how='inner')

#### Measures to Focus on

In [24]:
# list of measures to focus on
measures = ['CHD',
           'CASTHMA',
           'ACCESS2',
           'DEPRESSION',
           'DIABETES',
           'BPHIGH',
           'OBESITY',
           'STROKE']

#### Filter the DF by measures

In [25]:
tn_places_distances = tn_places_distances.loc[tn_places_distances['MeasureId'].isin(measures)]

In [26]:
# tn_places_distances.to_csv('../data/TN_Health_data_cleaned.csv')

In [27]:
# tn_places_distances['Measure'].unique()

#### 8 different measures

#### 'STROKE', 'OBESITY', 'CASTHMA', 'DEPRESSION', 'ACCESS2', 'DIABETES', 'BPHIGH', 'CHD'

#### 'Stroke among adults aged >=18 years',
#### 'Obesity among adults aged >=18 years',
#### 'Current asthma among adults aged >=18 years',
#### 'Depression among adults aged >=18 years',
#### 'Current lack of health insurance among adults aged 18-64 years',
#### 'Diagnosed diabetes among adults aged >=18 years',
#### 'High blood pressure among adults aged >=18 years',
#### 'Coronary heart disease among adults aged >=18 years'

In [28]:
tn_places_distances['distance'].agg(['min', 'max', 'median', 'mean'])

min           0.000000
max       52456.962023
median     7253.337776
mean       9573.433218
Name: distance, dtype: float64

### Distance to nearest parks KPIs
#### min           0.000000
#### max       52456.962023
#### median     7253.337776
#### mean       9573.433218

### Use quantile to create distance groups of equal num of  values

In [29]:
# Coonvert to KM
# Calculate the 1/3 and 2/3 quartile marks for distance and round to 2 decimal points
breakpoint = [
    round(tn_places_distances[['distance']].quantile(1/3).iloc[0] / 1000, 2),
    round(tn_places_distances[['distance']].quantile(2/3).iloc[0] / 1000, 2)
]
# Calculate the 1/6, 1/2, 5/6 quartile marks for distance and round to 2 decimal points
sub_break = [
    round(tn_places_distances[['distance']].quantile(1/6).iloc[0] / 1000, 2),
    round(tn_places_distances[['distance']].quantile(1/2).iloc[0] / 1000, 2),
    round(tn_places_distances[['distance']].quantile(5/6).iloc[0] / 1000, 2)
]

In [30]:
def create_q_groups(x):
    x = x / 1000
    if x <= breakpoint[0]:
        return 'close'
    elif x <= breakpoint[1]:
        return 'medium'
    else:
        return 'far'
    
def create_q_subs(x):
    x = x / 1000
    if x <= sub_break[0]:
        return f'0 km - {sub_break[0]} km'
    elif x <= breakpoint[0]:
        return f'{sub_break[0]} km - {breakpoint[0]} km'
    elif x <= sub_break[1]: 
        return f'{breakpoint[0]} km - {sub_break[1]} km'
    elif x <= breakpoint[1]: 
        return f'{sub_break[1]} km - {breakpoint[1]} km'
    elif x <= sub_break[2]: 
        return f'{breakpoint[1]} km - {sub_break[2]} km'
    else:
        return f'{sub_break[2]} km +'

In [31]:
tn_places_distances['distance_q_category'] = (tn_places_distances['distance']
                                              .apply(lambda r: create_q_groups(r)))
tn_places_distances['distance_q_sub'] = (tn_places_distances['distance']
                                              .apply(lambda r: create_q_subs(r)))
# tn_places_distances.to_csv('../data/TN_Health_data_cleaned.csv')

In [33]:
breakpoint

[3.96, 12.26]

In [34]:
sub_break

[1.39, 7.25, 17.78]